In [ ]:
from aiida_siesta.workflows.iterate import SiestaIterator
from aiida_siesta.workflows.converge import SiestaConverger, SiestaSequentialConverger
from aiida_siesta.workflows.base import SiestaBaseWorkChain
from aiida.orm.querybuilder import QueryBuilder
#from aiida_siesta.workflows.elastic import ElasticResponse
from aiida.engine import submit, run, CalcJob
from aiida.common import AttributeDict
from aiida.orm import Str, Float, Code, StructureData, Dict, List, KpointsData, Int, Bool, ArrayData, Group, BandsData
from aiida.orm.utils import load_node, load_group
from aiida.manage.database.delete.nodes import delete_nodes
from aiida_siesta.calculations.tkdict import FDFDict
import aiida
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import pathlib
import plotly.express as px
from xarray import DataArray

In [ ]:
def gather_outputs(node, as_df=False):
    import itertools

    if isinstance(node, (int, str)):
        node = load_node(node)

    df = pd.DataFrame([called.outputs.output_parameters.get_dict() for called in reversed(node.called)])
    
    inputs = {}
    for key, values in node.inputs.iterate_over.get_dict().items():
        values = [load_node(val) for val in values]
        try:
            values = [val.value for val in values]
            try:
                values = [float(val.split()[0]) for val in values]
            except:
                pass
        except:
            values = [str(val.pk) for val in values]
        inputs[key] = values
    
    if node.inputs.iterate_mode.value == "product":
        combs = list(itertools.product(*[val for val in inputs.values()]))
        
        indexes = pd.DataFrame(combs, columns=list(inputs.keys()))
    else:
        indexes = pd.DataFrame(inputs)
    
    df = df.set_index([indexes[col] for col in indexes])
    if not as_df:
        return df.to_xarray()
    else:
        return df.reset_index()

In [ ]:
aiida.load_profile('pfebrer')
siesta = Code.get_from_string('siesta@Laptop')

general_siesta_inputs = {
    "code": siesta, "pseudo_family": Str('NPG'), "parameters": Dict(),
    "options": Dict(dict={'withmpi': False,'resources': {'num_machines': 1, "num_mpiprocs_per_machine": 1},
                       'max_wallclock_seconds': 3600 * 2}),
}

The story of a convergence issue
----

- I asked Emanuele if I could implement a `Converger` in Aiida to learn about the framework.

- He said yes

- I noticed that most of the code to write was to **iterate over a given parameter**.

- I was stubborn enough to make Emanuele let me develop a standalone `Iterator` workchain.

- Now we have a general Iterator that can be used to **iterate over existing workchains** or **develop new workchains** that inherit from it.

And then we proposed it as an improvement for `aiida-core`.

3 weeks ago

<div style="color: darkred; font-weight: bold">No response yet... :(</div>

<center>
    <img src="https://thumbs.gfycat.com/DiligentEnlightenedFinnishspitz-max-1mb.gif">
</center>

BaseIterator
-------

In [ ]:
from aiida_siesta.workflows.iterate import BaseIterator

In [ ]:
from aiida_siesta.workflows.base import SiestaBaseWorkChain

class SiestaIterator(BaseIterator):
    _process_class = SiestaBaseWorkChain

**Additional parameters** can be made iterable by defining `_params_lookup`. You are not **limited to the inputs**!

But... you can already do

In [ ]:
for cutoff in [100,200,300]:
    submit(SiestaBaseWorkChain, **general_siesta_inputs, parameters={"mesh_cutoff": f"{cutoff} Ry"})

**... so what's the big deal?**

Ok, honestly it is not that big of a deal.

But it's **VERY PRACTICAL**.

SiestaIterator
-------

In [ ]:
from aiida_siesta.workflows.iterate import SiestaIterator

In [ ]:
SiestaIterator._params_lookup

Iterate over structures
--------

In [ ]:
import sisl
structures = [sisl.geom.graphene(bond=bond) for bond in (1.35, 1.40, 1.45, 1.50)]

In [ ]:
sisl.viz.SubPlots(plots=[struct.plot.bondlengthmap(axes=[0,1], sc=[2,2,1], points_per_bond=30) for struct in structures], arrange="square")

Convert to aiida's `StructureData`.

In [ ]:
structs = [StructureData(ase=struct.toASE()) for struct in structures]

And **run**!

In [ ]:
submit(SiestaIterator,
    # Regular SiestaBaseWorkchain inputs
    **general_siesta_inputs,
    # Iterator inputs
    iterate_over={"structure": structs},
)

But that's not everything!

The real high throughput
----



You have:
- `batch_size`: how many submissions in **parallel**?
- `iteration_mode`: how to iterate over **multiple parameters**.

In [ ]:
calc_node = submit(SiestaIterator,
    # Regular SiestaBaseWorkchain inputs
    **general_siesta_inputs,
                   
    # Iterator inputs
    iterate_over={
        "structure": structs, 
        "mesh_cutoff": ["50 Ry", "100 Ry"], 
        "paobasissize": ["SZP", "DZP"]
    },
    batch_size=Int(8),
    iterate_mode=Str("product")
)

In [ ]:
results = gather_outputs(calc_node, as_df=True)
results
px.line(results, x="mesh_cutoff", y=["E_KS"], color="paobasissize", facet_col="structure", facet_col_wrap=2)
#px.scatter_matrix(results, dimensions=["global_time", "mesh_cutoff", "paobasissize", "structure"], template="plotly")

BaseConverger
----------------

`BaseIterator` has a `_should_proceed` method that can be overwritten.

One can easily use it to **implement a converger**.

Then, `SiestaConverger` is nothing more than:

```python
class SiestaConverger(BasicConverger, SiestaIterator):
    pass
```

SiestaConverger
------

In [ ]:
from aiida_siesta.workflows.converge import SiestaConverger



Currently, it has 2 additional inputs:
- `target`, *the **name of the output parameter** to check for convergence*.
- `threshold`, *the **difference between two steps** to consider the calculation converged*

In [ ]:
calc_node = submit(SiestaConverger,
    # Regular SiestaBaseWorkchain inputs
    **general_siesta_inputs,
    structure=structs[2],
                   
    # Iterator inputs
    batch_size=Int(2),
    iterate_over={"kpoints_0": [1,3,6,9,12]},
                   
    # Converger inputs
    target=Str("E_KS"),
    threshold=Float(0.05)
    
)

In [ ]:
calc_node.outputs.converged_target_value

SiestaSequentialConverger 
----

In [ ]:
from aiida_siesta.workflows.converge import SiestaSequentialConverger

In [ ]:
calc_node = submit(SiestaSequentialConverger,
    
    # Sequential converger inputs
    iterate_over=[
        {"mesh_cutoff": ["50 Ry", "100 Ry", 
                         "150 Ry", "200 Ry"]},
        {"kpoints_0": [1,3,6,9,12]},
        {"kpoints_1": [1,3,6,9,12]},
    ],
    
    converger_inputs=dict(
        # Regular SiestaBaseWorkchain inputs
        **general_siesta_inputs,
        structure=structs[2],
        
        # Converger inputs
        batch_size=Int(4),
        target=Str("E_KS"),
        threshold=Float(0.05)
    )  
)

<center>
<img src="./diagram.svg"/>
</center>

In [ ]:
calc_node.called[0].inputs.kpoints.get_kpoints_mesh()

*Thank you*.